In [1]:
import requests
import pandas as pd
import json

import sys
import os

sys.path.append(os.path.abspath('queries'))
from queries import *

# Met
source: csv file. We read it as pandas dataframe, count how many ids with the desired characteristics

In [2]:
met_path = "downloaded-data/MetObjects.txt"
met_df = pd.read_csv(met_path, delimiter=',', dtype=str)
met_df.head()

,Object Number,Is Highlight,Is Timeline Work,Is Public Domain,Object ID,Gallery Number,Department,AccessionYear,Object Name,Title,...,River,Classification,Rights and Reproduction,Link Resource,Object Wikidata URL,Metadata Date,Repository,Tags,Tags AAT URL,Tags Wikidata URL
0,1979.486.1,False,False,False,1,NaN,The American Wing,1979,Coin,One-dollar Liberty Head Coin,...,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/1,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
1,1980.264.5,False,False,False,2,NaN,The American Wing,1980,Coin,Ten-dollar Liberty Head Coin,...,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/2,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
2,67.265.9,False,False,False,3,NaN,The American Wing,1967,Coin,Two-and-a-Half Dollar Coin,...,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/3,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
3,67.265.10,False,False,False,4,NaN,The American Wing,1967,Coin,Two-and-a-Half Dollar Coin,...,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/4,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
4,67.265.11,False,False,False,5,NaN,The American Wing,1967,Coin,Two-and-a-Half Dollar Coin,...,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/5,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN


In [3]:
columns_to_keep = ["Object Number", "Culture", "Period", "Dynasty", "Reign", "Object Date", "Object Begin Date", "Object End Date", "Artist Alpha Sort", "Artist Nationality", "Artist Begin Date", "Artist End Date", "Object Wikidata URL", "Artist ULAN URL", "Artist Wikidata URL"]

In [4]:
filtered_df = met_df[columns_to_keep]
duplicated_mask = filtered_df['Object Number'].duplicated(keep=False)

# Invert the mask to keep only unique values
unique_values_df = filtered_df[~duplicated_mask]
unique_values_df = unique_values_df.replace("NaN", " ")
unique_values_df.head(10)

,Object Number,Culture,Period,Dynasty,Reign,Object Date,Object Begin Date,Object End Date,Artist Alpha Sort,Artist Nationality,Artist Begin Date,Artist End Date,Object Wikidata URL,Artist ULAN URL,Artist Wikidata URL
0,1979.486.1,NaN,NaN,NaN,NaN,1853,1853,1853,"Longacre, James Barton",American,1794,1869,NaN,http://vocab.getty.edu/page/ulan/500011409,https://www.wikidata.org/wiki/Q3806459
2,67.265.9,NaN,NaN,NaN,NaN,1909–27,1909,1927,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,67.265.10,NaN,NaN,NaN,NaN,1909–27,1909,1927,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,67.265.11,NaN,NaN,NaN,NaN,1909–27,1909,1927,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,67.265.12,NaN,NaN,NaN,NaN,1909–27,1909,1927,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,67.265.13,NaN,NaN,NaN,NaN,1909–27,1909,1927,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,67.265.14,NaN,NaN,NaN,NaN,1909–27,1909,1927,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,67.265.15,NaN,NaN,NaN,NaN,1909–27,1909,1927,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1979.486.3,NaN,NaN,NaN,NaN,1912,1912,1912,"Pratt, Bela Lyon",American,1867,1917,NaN,http://vocab.getty.edu/page/ulan/500095555,https://www.wikidata.org/wiki/Q4881787
10,1979.486.2,NaN,NaN,NaN,NaN,1907,1907,1907,"Gobrecht, Christian",American,1785,1844,NaN,http://vocab.getty.edu/page/ulan/500077295,https://www.wikidata.org/wiki/Q5109648


In [ ]:
#filtered_df = unique_values_df[unique_values_df["Dynasty"].notna() & (unique_values_df["Dynasty"] != '')]
#filtered_df.head(10)

In [5]:
met_art_tot = unique_values_df["Object Number"].notna().sum() - (unique_values_df["Object Number"] == '').sum()
print(met_art_tot)

478366


In [11]:
def make_prop_list(res_list): 
    "returns the dataset proportion list. Total of artwork at index one, name of dataset at index 0" 
    # append the name of the dataset to the prop_list
    l_prop = [res_list[0]]
    tot = res_list[1]
    for num in res_list[2:]:
        p = round((int(num)/int(tot))*100, 2)
        l_prop.append(p)
  
    return l_prop

In [16]:
n = 0
date_columns = ["Object Date", "Object Begin Date", "Object End Date"]
artist = ["Artist Alpha Sort"]
artist_date_columns = ["Artist Begin Date", "Artist End Date"]
art_aligned = ["Object Wikidata URL", "Artist ULAN URL", "Artist Wikidata URL"]

columns_names = [date_columns, artist, artist_date_columns, art_aligned]
date_tot = 0
artist_tot = 0
artist_date_tot = 0
aligned_tot = 0

count = 0 
for index, row in unique_values_df.iterrows():
    if count < 20: 
        obj = row["Object Number"]
        print("------------ Object: ", obj)
        date = None
        artist = None
        artist_date = None
        align = None
        for l in columns_names:
            i = columns_names.index(l)
            for name in l:
                cell_value = row[name]
                if pd.isna(cell_value) or cell_value == '':
                    print("field ", name, " empty cell")
                else: 
                    print("field ", name, "has value ", cell_value)
                    if i == 0:
                        date = True
                    if i == 1:
                        artist = True
                    if i == 2:
                        artist_date = True
                    if i == 3:
                        align = True
        if date == True: 
            date_tot += 1
        if artist == True:
            artist_tot += 1
        if artist_date == True:
            artist_date_tot += 1
        if align == True:
            aligned_tot += 1
    count+=1            
res_list = ["Tate", met_art_tot, date_tot, 0, 0, artist_tot, artist_date_tot, 0, aligned_tot]
prop_list = make_prop_list(res_list)
print(res_list)
print(prop_list)

------------ Object:  1979.486.1
field  Object Date has value  1853
field  Object Begin Date has value  1853
field  Object End Date has value  1853
field  Artist Alpha Sort has value  Longacre, James Barton
field  Artist Begin Date has value  1794      
field  Artist End Date has value  1869      
field  Object Wikidata URL  empty cell
field  Artist ULAN URL has value  http://vocab.getty.edu/page/ulan/500011409
field  Artist Wikidata URL has value  https://www.wikidata.org/wiki/Q3806459
------------ Object:  67.265.9
field  Object Date has value  1909–27
field  Object Begin Date has value  1909
field  Object End Date has value  1927
field  Artist Alpha Sort  empty cell
field  Artist Begin Date  empty cell
field  Artist End Date  empty cell
field  Object Wikidata URL  empty cell
field  Artist ULAN URL  empty cell
field  Artist Wikidata URL  empty cell
------------ Object:  67.265.10
field  Object Date has value  1909–27
field  Object Begin Date has value  1909
field  Object End Date has

In [23]:
# Define column groups
date_columns = ["Object Date", "Object Begin Date", "Object End Date"]
artist_columns = ["Artist Alpha Sort"]
artist_date_columns = ["Artist Begin Date", "Artist End Date"]
art_aligned = ["Object Wikidata URL"]
artist_aligned_columns = ["Artist ULAN URL", "Artist Wikidata URL"]

# Function to count non-empty cells
def count_non_empty(df, columns):
    return df[columns].notna().sum(axis=1).gt(0).sum()

# Count non-empty cells in each group
date_tot = count_non_empty(unique_values_df, date_columns)
artist_tot = count_non_empty(unique_values_df, artist_columns)
artist_date_tot = count_non_empty(unique_values_df, artist_date_columns)
art_aligned_tot = count_non_empty(unique_values_df, art_aligned)
artist_aligned_tot = count_non_empty(unique_values_df, artist_aligned_columns)

#print(f"date_tot: {date_tot}, artist_tot: {artist_tot}, artist_date_tot: {artist_date_tot}, aligned_tot: {art_aligned_tot}")

res_list = ["Tate", int(met_art_tot), int(date_tot), 0, 0, int(artist_tot), int(artist_date_tot), 0, int(art_aligned_tot), int(artist_aligned_tot)]
prop_list = make_prop_list(res_list)
print(res_list)
print(prop_list)

# ['Tate', 478366, np.int64(478366), 0, 0, np.int64(278089), np.int64(278089), 0, np.int64(268576)]
#['Tate', 100.0, 0.0, 0.0, 58.13, 58.13, 0.0, 56.14]

['Tate', 478366, 478366, 0, 0, 278089, 278089, 0, 68024, 224447]
['Tate', 100.0, 0.0, 0.0, 58.13, 58.13, 0.0, 14.22, 46.92]


In [25]:
headers = ['Dataset', 'Artworks', 'With date', 'With place', 'With date and place', 'With author', 'With author with date', 'With author with place','With artworks aligned to Wikidata', 'With author aligned to Wikidata or Ulan']
headers_prop = headers.copy()
headers_prop.remove('Artworks')

In [26]:
import csv

with open('met_res.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(headers)
    writer.writerow(res_list)
with open('met_res_prop.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(headers_prop)
    writer.writerow(prop_list)

In [ ]:
import requests

def analyze_met_collection():
    endpoint = "https://collectionapi.metmuseum.org/public/collection/v1/objects"
    response = requests.get(endpoint)
    data = response.json()

    object_ids = data['objectIDs'][:1000]  # Analyze a subset for demonstration

    date_present = 0
    place_present = 0

    for obj_id in object_ids:
        obj_response = requests.get(f"{endpoint}/{obj_id}")
        obj_data = obj_response.json()

        if 'objectBeginDate' in obj_data and 'objectEndDate' in obj_data:
            date_present += 1
        if 'repository' in obj_data:  # Adjust this field based on actual data structure
            place_present += 1

    total_objects = len(object_ids)
    print(f"Total objects analyzed: {total_objects}")
    print(f"Objects with date of production: {date_present} ({(date_present / total_objects) * 100:.2f}%)")
    print(f"Objects with place of production: {place_present} ({(place_present / total_objects) * 100:.2f}%)")

analyze_met_collection()


# Tate
Documentation: https://public.opendatasoft.com/api/explore/v2.1/console 
both exports and downloads limited to 500 entries. 
Alternative way: https://github.com/tategallery/collection/tree/master 
https://github.com/SPARQL-Anything/showcase-tate

In [34]:
tate_api = "https://public.opendatasoft.com/api/explore/v2.1/catalog/exports"
tate_export = "https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/the-tate-collection/exports/json?limit=-1&timezone=Europe%2FBerlin&use_labels=false&epsg=4326"
tate_id = "the-tate-collection"
tate_export2 = "https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/the-tate-collection/exports/json?limit=-1"


In [35]:

req = requests.get(tate_export2)
tate_data2 = req.json()

In [36]:
tate_store2 = store_in_json("tate_data2.json", tate_data2)

In [43]:
# open download

import json
f = open('downloaded-data/the-tate-collection.json')

# returns JSON object as 
# a dictionary
tate_3 = json.load(f)


In [44]:
print(len(tate_3))

500


In [14]:
limit = 0
headings = []
for d in tate_data: 
    if limit < 50: 
        for k in d.keys(): 
            if k not in headings: 
                headings.append(k)
    limit+=1
print(headings)

['id', 'accession_number', 'artist', 'artistrole', 'artistid', 'title', 'datetext', 'medium', 'creditline', 'year', 'acquisitionyear', 'dimensions', 'width', 'height', 'depth', 'units', 'inscription', 'thumbnailcopyright', 'thumbnailurl', 'url']


In [63]:
test = None 
if str(test) == "None": 
    print("ok")

ok


In [66]:
# identify null values (entry, but not known/negative entry)
null_values = ["date not known", "None", "null", "Null"]

In [110]:
import os
import json
import time

def artist_date_place(base_path):
    with_date = set()
    with_place = set()
    count_files = 0
    files_crashed = 0
    # Traverse the directory structure

    start = time.time()
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.endswith('.json'):
                count_files+=1
                try:
                    with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
                        data = json.load(f)
                        # Check if there is the id: 
                        if "id" in data and str(data['id']) not in null_values: 
                            artist_id = data['id']
                            artist_name = data['fc']
                            #print("------------artist: ", artist_name, artist_id)
                            # set details presence to None
                            date_info = None
                            place_info = None
                            if 'birth' in data: 
                                if 'place' in data['birth']:
                                    birth_place = data['birth']['place']
                                    if 'name' in birth_place and str(birth_place['name']) not in null_values:
                                       # print("birth place: ", birth_place['name'])
                                        place_info = True
                                if 'time' in data['birth']: 
                                    birth_time = data['birth']['time']
                                    if 'startYear' in birth_time and str(birth_time['startYear']) not in null_values: 
                                        date_info = True
    
                            if place_info == None and 'death' in data: 
                                if 'place' in data['death']:
                                    death_place = data['death']['place']
                                    if 'name' in death_place and str(death_place['name']) not in null_values:
                                       # print("death place: ", death_place['name'])
                                        place_info = True
                                        
                            if 'activePlaces' in data:
                                for d in data['activePlaces']:
                                    if place_info == None: 
                                        if 'name' in d and d['name'] not in null_values:
                                            place_info = True
                                        
                            if "date" in data and str(data["date"]) not in null_values: 
                               # print("date: ", data["date"])
                                date_info = True
                            
                                
                            # add artists with details to the respective set
                            if place_info == True: 
                                with_place.add(artist_id) 
                            if date_info == True: 
                                with_date.add(artist_id)                           
                                
                except Exception as e: 
                    print(f"Error reading {file}: {e}")
                    files_crashed+=1
                    
                print("files processed: ", count_files)
    end = time.time()
    print("tot files processed: ", count_files, "tot files crashed: ", files_crashed)
    print("tot artists with date: ", len(with_date), "tot artists with place: ", len(with_place))

    t = end -start
    print("Total time: ", t)
    return with_place, with_date

artists_path = "downloaded-data/collection-master/artists"
artist_test_path = "downloaded-data/collection-master/artists/b"


# Count the total number of artworks
artists_with_place, artists_with_date = artist_date_place(artists_path)

print(artists_with_place)


files processed:  1
files processed:  2
files processed:  3
files processed:  4
files processed:  5
files processed:  6
files processed:  7
files processed:  8
files processed:  9
files processed:  10
files processed:  11
files processed:  12
files processed:  13
files processed:  14
files processed:  15
files processed:  16
files processed:  17
files processed:  18
files processed:  19
files processed:  20
files processed:  21
files processed:  22
files processed:  23
files processed:  24
files processed:  25
files processed:  26
files processed:  27
files processed:  28
files processed:  29
files processed:  30
files processed:  31
files processed:  32
files processed:  33
files processed:  34
files processed:  35
files processed:  36
files processed:  37
files processed:  38
files processed:  39
files processed:  40
files processed:  41
files processed:  42
files processed:  43
files processed:  44
files processed:  45
files processed:  46
files processed:  47
files processed:  48
f

In [111]:
import os
import json
import time





def count_artworks(base_path):
    artwork_count = 0
    tate_art_date_count = 0
    tate_art_place_count = 0
    tate_artist_count = 0
    tate_artist_date_count = 0
    tate_artist_place_count = 0

    start = time.time()
    count_files = 0
    files_crashed = 0
    # Traverse the directory structure
    for root, dirs, files in os.walk(base_path):
        tate_artwork_count = 0
        for file in files:
            if file.endswith('.json'):
                count_files +=1
                art_has_auth_place = None
                art_has_auth_date = None
                try:
                    with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
                        data = json.load(f)
                        # Check if it's an artwork file
                        if 'id' in data and 'title' in data:
                          #  print("---------- Artwork: ", data["title"], data["id"])
                            artwork_count += 1
                        if 'dateText' in data and str(data['dateText']) not in null_values: 
                           # print("artwork date: ", data['dateText'])
                            tate_art_date_count+=1
                        if 'all_artists' in data and str(data['all_artists']) not in null_values:
                            tate_artist_count+=1
                            artist_details = data['contributors']

                            for d in artist_details: 
                                if art_has_auth_place == None or art_has_auth_date == None: 
                                    if 'id' in d and str(d['id']) not in null_values: 
                                        artist_id = d['id']
                                        if artist_id in artists_with_place: 
                                          #  print("artist has place: ", artist_id)
                                            art_has_auth_place = True
                                        if artist_id in artists_with_date: 
                                          #  print("artist has date: ", artist_id)
                                            art_has_auth_date = True
                                        
                                       
                        if art_has_auth_date == True: 
                            tate_artist_date_count+=1
                        if art_has_auth_place == True: 
                            tate_artist_place_count+=1
                                
                except Exception as e: 
                    print(f"Error reading {file}: {e}")
                    files_crashed+=1
                print("files processed: ", count_files)

    end = time.time()
    t = end - start
    print("total time: ", t)
    print("files processed: ", count_files, " File crashed: ", files_crashed)
    result_list = ["Tate", artwork_count, tate_art_date_count, tate_art_place_count, 0, tate_artist_count, tate_artist_date_count, tate_artist_place_count, 0, 0, 0]
    print("Result: list with artwork_count, tate_art_date_count, tate_art_place_count, tate_artist_count, tate_artist_date_count, tate_artist_place_count, artist wd, artist ulan")

    count_list = ["Tate", artwork_count]
    prop_list = ["Tate"]
    results = result_list[2:]
    for number in results: 
        p = pprint_prop(number, artwork_count, count_list, prop_list)
    print("count_list: ", count_list)
    print("prop_list: ", prop_list)
    return count_list, prop_list

# Define the path to the 'artworks' folder
tate_artworks_path = "downloaded-data/collection-master/artworks"

# Count the total number of artworks
tate_count, tate_prop = count_artworks(tate_artworks_path)


files processed:  1
files processed:  2
files processed:  3
files processed:  4
files processed:  5
files processed:  6
files processed:  7
files processed:  8
files processed:  9
files processed:  10
files processed:  11
files processed:  12
files processed:  13
files processed:  14
files processed:  15
files processed:  16
files processed:  17
files processed:  18
files processed:  19
files processed:  20
files processed:  21
files processed:  22
files processed:  23
files processed:  24
files processed:  25
files processed:  26
files processed:  27
files processed:  28
files processed:  29
files processed:  30
files processed:  31
files processed:  32
files processed:  33
files processed:  34
files processed:  35
files processed:  36
files processed:  37
files processed:  38
files processed:  39
files processed:  40
files processed:  41
files processed:  42
files processed:  43
files processed:  44
files processed:  45
files processed:  46
files processed:  47
files processed:  48
f

In [112]:
headers = ['Dataset', 'Artworks', 'With date', 'With place', 'With date and place', 'With author', 'With author with date', 'With author with place','With artworks aligned to Wikidata', 'With author aligned to Wikidata', 'With author aligned to ULAN']
headers_prop = headers.copy()
headers_prop.remove('Artworks')

In [ ]:
res = [headers, tate_count]
res_prop = [headers_prop, tate_prop]

In [113]:
import csv

with open('tate_res.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(headers)
    writer.writerow(tate_count)
with open('tate_res_prop.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(headers_prop)
    writer.writerow(tate_prop)

# Smithsonian

https://edan.si.edu/openaccess/apidocs/ 

API limited to 10 results. for developers: github repo: https://github.com/Smithsonian/OpenAccess 

In [48]:
smithsonian_key = "dcrhkyiESFQALOO375POc3Cxl4IU7gPGNKpBXLFk" 

In [60]:
sm_test= "https://api.si.edu/openaccess/api/v1.0/search?api_key=dcrhkyiESFQALOO375POc3Cxl4IU7gPGNKpBXLFk&rows=-1&q=*"
req = requests.get(sm_test)
sm_res = req.json()

In [61]:
count = 0
dict_list = sm_res['response']['rows']
print(len(dict_list))

10
